<a href="https://colab.research.google.com/github/KimGriezmann/DeepLearning_BeTheLegend/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%20%ED%8C%8C%EC%9D%BC/%EC%B5%9C%EA%B7%BC%205%EC%9D%BC(%EB%B3%80%EA%B2%BD%20%EA%B0%80%EB%8A%A5)%20%EA%B2%BD%EA%B8%B0%EC%9D%98%20%ED%83%80%EC%9C%A8%20%EA%B5%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import tqdm

In [4]:
data = pd.read_excel("/content/날짜별 타자 데이터 통합 (1).xlsx")
hitter = pd.read_excel("/content/90명 타자 명단.xlsx")

In [5]:
data = pd.DataFrame(data)
hitter = pd.DataFrame(hitter)

In [6]:
print(data.head(), len(data))

     day name  vs  타순  P  ab  hit  ...  SSG   KT        KW  KIA   HH   SS   NC
0  44289  강민호  키움   6  C   3    0  ...  0.0  0.0  0.000000  0.0  0.0  0.0  0.0
1  44290  강민호  키움   7  C   4    2  ...  0.0  0.0  0.285714  0.0  0.0  0.0  0.0
2  44292  강민호  두산   9  C   1    0  ...  0.0  0.0  0.000000  0.0  0.0  0.0  0.0
3  44293  강민호  두산   6  C   4    1  ...  0.0  0.0  0.000000  0.0  0.0  0.0  0.0
4  44294  강민호  두산   7  C   3    2  ...  0.0  0.0  0.000000  0.0  0.0  0.0  0.0

[5 rows x 25 columns] 5296


In [7]:
print(hitter.head(), len(hitter))

  Team batter
0   롯데    전준우
1   롯데    손아섭
2   롯데     정훈
3   롯데    마차도
4   롯데    한동희 90


In [8]:
data.loc[:,['day','name','ab','hit']].dtypes

day      int64
name    object
ab       int64
hit      int64
dtype: object

In [9]:
# 더블헤더 처리
temp = np.zeros(len(data))

# day, name이 같은 값이 있다는 것은 더블헤더 경기가 있다는것
# keep=False 옵션은 중복 값에 해당하는 튜플들을 모두 True로 리턴, 그 외 옵션 : 'first', 'last' 의 경우 둘 중 하나만 True
dup = data.loc[:,['day','name']].duplicated(keep=False) 

for i in range(len(data)):
  if dup[i]==False:
    temp[i]=0
  elif dup[i]==True:
    temp[i]=1

# 중복의 첫번째 행은 더블헤더 1차전
# 중복인 두번째 행이 더블헤더 2차전이다.
for i in range(1,len(temp)):
  if temp[i]==1 and temp[i-1]==1:
    temp[i]=2

data['double']=temp

In [10]:
#
# 선수마다 최근 5일의 타율을 구해야함. 
# 현재 day에서 -5부터 -1까지의 sum(hit)/sum(ab) 값을 구하면 된다

# hitter_data : 90명 타자 data
# record_data : 날짜별 타자 기록
# recent_day : 기본(최근 5일), 설정 가능
def make_recent_days(hitter_data, record_data, day=5):

  # 최근 5일(기본) 경기 타율 저장할 공간 생성
  recent_days_avg = np.zeros(len(record_data)) # 최근 5일(기본) 경기 타율
  recent_days_hitter = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  recent_days_day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜
  recent_days_double = np.zeros(len(record_data), dtype=np.int8) # 더블헤더 분류 / 0 : 더블헤더 아님 / 1 : 더블헤더 1차전 / 2 : 더블헤더 2차전

  ps = 0
  for h in tqdm.tqdm(hitter_data['batter']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h].reset_index() # 특정 타자 한명에 대해서만 데이터 추출
    for i in range(len(hdata['day'])): # 추출한 데이터에서 한 행씩 접근
      recent_days_hitter[ps] = h # 현재 타자
      r = hdata.loc[i,'day']  # 현재 기준 날짜
      recent_days_day[ps] = r
      d = hdata.loc[i,'double'] # 현재 더블헤더 분류
      recent_days_double[ps] = d      
      if d==2: # 더블헤더 2차전이라면
        hrdata = hdata.loc[hdata['day']>=r-day].loc[hdata['day']<=r].loc[hdata['double']!=2] # r 날짜까지 가져와서 2차전을 제외
      else: # 더블헤더가 아니고, 더블헤더 1차전이라면
        hrdata = hdata.loc[hdata['day']>=r-day].loc[hdata['day']<=r-1] # 접근한 행의 날짜를 기준으로 5일 경기
         
      if len(hrdata)==0: # 최근 5일 경기 없으면 넘어가고
        ps+=1
        continue
      else : # 있으면
        sum_ab = sum(hrdata['ab'])
        if sum_ab==0: # 경기 기록은 존재하지만 타수(ab)가 0
          ps+=1
          continue
        sum_hit = sum(hrdata['hit'])
        recent_days_avg[ps] = sum_hit/sum_ab
        ps+=1

  result_Dataframe = pd.DataFrame()
  result_Dataframe['day'] = recent_days_day
  result_Dataframe['name'] = recent_days_hitter
  result_Dataframe['recent_days_avg'] = recent_days_avg
  result_Dataframe['double'] = recent_days_double

  return result_Dataframe

In [11]:
recent_days_data = make_recent_days(hitter, data)

100%|██████████| 90/90 [00:11<00:00,  7.76it/s]


In [12]:
recent_days_data.loc[recent_days_data['day']==44373].loc[recent_days_data['name']=='강민호']

,day,name,recent_days_avg,double
1847,44373,강민호,0.333333,1
1848,44373,강민호,0.352941,2


In [13]:
data.loc[data['name']=='강민호',:].loc[54:59,:]

,day,name,vs,타순,P,ab,hit,vs_cu_ab,vs_cu_hit,result,avg,slg,recent_5,recent_10,success_recent_10,LT,OB,LG,SSG,KT,KW,KIA,HH,SS,NC,double
54,44367,강민호,롯데,5,C,4,1,5,2,1,0.332,0.500,0.157895,0.242,0.5,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
55,44369,강민호,한화,4,C,4,1,4,1,1,0.330,0.495,0.157895,0.212,0.5,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.300000,0.0,0.0,0.0
56,44370,강민호,한화,4,C,4,2,2,1,1,0.333,0.495,0.210526,0.235,0.6,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0
57,44371,강민호,한화,4,C,4,1,4,0,1,0.332,0.505,0.315789,0.235,0.6,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.321429,0.0,0.0,0.0
58,44373,강민호,LG,4,C,5,2,4,1,1,0.333,0.517,0.350000,0.243,0.7,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0
59,44373,강민호,LG,8,C,1,1,5,3,1,0.337,0.520,0.333333,0.250,0.7,0.000000,0.0,0.360000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.0


In [14]:
print((1+2+1)/(4+4+4))
print((1+2+1+2)/(4+4+4+5))

0.3333333333333333
0.35294117647058826
